In [3]:
import numpy as np

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow import keras

In [48]:
def convLSTM(input_img):
    convLSTM1 = ConvLSTM2D(filters=128, kernel_size=(3, 3),
                   padding='same', data_format='channels_last', return_sequences=True, 
                           activation='relu')(input_img)
    
    convtd1 = TimeDistributed((Conv2D(128, kernel_size=3, padding = "same", activation = 'relu')))(convLSTM1)
    
    mptd1 = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(convtd1)
    
    convLSTM2 = ConvLSTM2D(filters=256, kernel_size=(3, 3),
                   padding='same', data_format='channels_last', return_sequences=True, 
                           activation='relu')(mptd1)
    
    convtd2 = TimeDistributed((Conv2D(256, kernel_size=3, padding = "same", activation = 'relu')))(convLSTM2)
    
    mptd2 = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(convtd2)
    
    convtd3 = TimeDistributed(Conv2D(512, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(mptd2)
    convtd4 = TimeDistributed(Conv2D(512, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd3)
    
    
    upconvtd1 = TimeDistributed(Conv2DTranspose(256, kernel_size=2, strides=2, 
                                                padding='same',kernel_initializer = 'he_normal'))(convtd4)
    
    # concatenate convtd4
    merge1 = concatenate([convtd2, upconvtd1]) # 512 kernel
    
    convtd5 = TimeDistributed(Conv2D(256, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(merge1)
    convtd6 = TimeDistributed(Conv2D(256, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd5)
    
    upconvtd2 = TimeDistributed(Conv2DTranspose(128, kernel_size=2, strides=2, 
                                        padding='same',kernel_initializer = 'he_normal'))(convtd6)
    
    # concatenate convtd2
    merge2 = concatenate([convtd1, upconvtd2])
    
    convtd7 = TimeDistributed(Conv2D(128, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(merge2)
    convtd8 = TimeDistributed(Conv2D(128, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd7)
    
    convtd9 = TimeDistributed(Conv2D(1, kernel_size=3, activation = 'sigmoid', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd8)
    
    return convtd9

In [49]:
input_img = Input(shape = (16, 128, 128, 3))

autoencoder = Model(input_img, convLSTM(input_img))

# autoencoder.compile(loss=keras.losses.binary_crossentropy, optimizer='adam')

In [50]:
autoencoder.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 16, 128, 128 0                                            
__________________________________________________________________________________________________
conv_lst_m2d_34 (ConvLSTM2D)    (None, 16, 128, 128, 604160      input_18[0][0]                   
__________________________________________________________________________________________________
time_distributed_164 (TimeDistr (None, 16, 128, 128, 147584      conv_lst_m2d_34[0][0]            
__________________________________________________________________________________________________
time_distributed_165 (TimeDistr (None, 16, 64, 64, 1 0           time_distributed_164[0][0]       
___________________________________________________________________________________________